In [1]:
%%configure -f
{"executorCores": 4, "executorMemory": "47969M", "conf":{"spark.task.cpus": 4}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1565254661534_0002,pyspark,idle,Link,Link,


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, SoupStrainer
import html2text
from smart_open import smart_open
from time import time
from tqdm import tqdm
import concurrent.futures

import json

from itertools import islice

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

import scrapy
from scrapy.linkextractors import LinkExtractor
import html2text
from urllib.parse import urlparse, ParseResult
from scrapy.linkextractors import IGNORED_EXTENSIONS
from scrapy.selector import Selector

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1565254661534_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
input_file = "s3://onai-ml-dev-eu-west-1/web_crawler/data/seed_urls/company-urls-by-country.csv"

def extract_text_from_html(html, upper_bound=10000):
    if len(html.split("\n")) > upper_bound:
        return ""
    parser = html2text.HTML2Text()
    parser.wrap_links = False
    parser.skip_internal_links = True
    parser.inline_links = True
    parser.ignore_anchors = True
    parser.ignore_images = True
    parser.ignore_emphasis = True
    parser.ignore_links = True
    return parser.handle(html)

def convert_url(url):
    p = urlparse(url, 'http')
    netloc = p.netloc or p.path
    path = p.path if p.netloc else ''
    if not netloc.startswith('www.'):
        netloc = 'www.' + netloc

    p = ParseResult('http', netloc, path, *p[3:])
    return p.geturl()

class MySpider(scrapy.Spider):
    name = "my_spider"

    # crawl in BFS
    custom_settings = {
        'DEPTH_LIMIT': 2,
        'DEPTH_PRIORITY': 1,
        'SCHEDULER_DISK_QUEUE': 'scrapy.squeues.PickleFifoDiskQueue',
        'SCHEDULER_MEMORY_QUEUE': 'scrapy.squeues.FifoMemoryQueue',
        'SCHEDULER_PRIORITY_QUEUE': 'scrapy.pqueues.DownloaderAwarePriorityQueue',
        'DNS_TIMEOUT': 20,
        'DOWNLOAD_TIMEOUT': 30,
        'LOG_LEVEL': 'ERROR',
        'CONCURRENT_REQUESTS': 100,
        'REACTOR_THREADPOOL_MAXSIZE': 20,        
        'REDIRECT_ENABLED': False,
        'RETRY_ENABLED': False,
    }

    def __init__(self, *args, **kwargs):
        start_urls_and_ids = kwargs.get('start_urls_and_ids')
        start_urls = [start_url for _,start_url in start_urls_and_ids]

        parsed_urls = [urlparse(start_url) for start_url in start_urls]
        url_domains = [parsed_url.netloc for parsed_url in parsed_urls]
        
        self.company_ids = {start_url:company_id for company_id,start_url in start_urls_and_ids}
        self.allowed_domains = url_domains
        
        print(self.company_ids)

        self.output = kwargs.get('output')

        self.allowed_extensions = ('.pdf', '.doc', '.xls', '.docx', '.xlsx')
        self.denied_extensions = tuple(set(['.' + ext for ext in IGNORED_EXTENSIONS]) - set(self.allowed_extensions))

        super(MySpider, self).__init__(*args, **kwargs, start_urls=start_urls)

    def save_file(self, response, upper_limit=2000000): # ~ 2 mbs tops
        page_url = response.meta.get('page_url')
        root_page = response.meta.get('root_page')
        company_id = self.company_ids[root_page]
        
        path = response.url.split('/')[-1]
        filename = path[path.rfind("/") + 1:]
        if len(response.body) < upper_limit:
            self.output.append((company_id, 
                                root_page, 
                                description, 
                                page_url, 
                                filename, 
                                "file", 
                                bytearray(response.body)))

    def parse(self, response):
        title_xpath = response.selector.xpath('//title/text()').extract()
        page_title = title_xpath[0] if title_xpath else ""
        
        root_page = response.meta.get("root_page")
        if root_page is None:
            root_page = response.url
            
        company_id = self.company_ids[root_page]

        description = response.meta.get("root_description")
        if description is None:
            description_xpath = response.selector.xpath('//meta[@name=\'description\']/@content').extract()
            description = description_xpath[0] if description_xpath else ""

        page_content = extract_text_from_html(response.text)
        self.log('Page: %s (%s)' % (response.url, page_title))

        extracted_links = LinkExtractor().extract_links(response)
        for link in extracted_links:
            url = link.url
            parsed_url = urlparse(url)
            url_domain = parsed_url.netloc
            
            follow_link = False
            for marker in ['who we are', 'overview', 'about', 'mission']:
                if marker in link.url.lower():
                    follow_link = True

            for marker in ['who we are', 'overview', 'about', 'mission']:
                if marker in link.text.lower():
                    follow_link = True
                    
            if not follow_link:
                continue

            if url_domain in self.allowed_domains:
                if link.url.endswith(self.allowed_extensions):
                    yield scrapy.Request(url,
                                         callback=self.save_file,
                                         meta={'page_url': response.url, 
                                               'root_page':root_page,
                                               'root_description': description
                                              }
                                        )
                elif link.url.endswith(self.denied_extensions):
                    continue
                else:
                    yield scrapy.Request(url,
                                         callback=self.parse,
                                         meta={'root_page': root_page,
                                               'root_description': description
                                              }
                                        )
        
        self.output.append((company_id,
                            root_page,
                            description,
                            response.url,
                            page_title,
                            "html",
                            page_content))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
spark.read.csv(input_file, header=True, inferSchema=True, sep="\t").count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7309166

In [4]:
df = (spark.read.csv(input_file, header=True, inferSchema=True, sep="\t")
      .withColumnRenamed("web_page", "url")
      .filter(F.col("url").isNotNull())
      .repartition(1000)
      .distinct()
     )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1430447

In [9]:
import logging
logging.basicConfig(level=logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [5]:
df.write.parquet("s3://onai-ml-dev-eu-west-1/web_crawler/data/seed_urls/company_urls.parquet", mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
convert_url_udf = F.udf(convert_url, T.StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df = spark.read.load("s3://onai-ml-dev-eu-west-1/web_crawler/data/seed_urls/company_urls.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- company_id: integer (nullable = true)
 |-- company_name: string (nullable = true)
 |-- url: string (nullable = true)
 |-- country: string (nullable = true)

In [31]:
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1430447

In [10]:
def run_spider(urls):
    from twisted.internet import reactor
    from scrapy.crawler import CrawlerRunner
    
    from crochet import setup, wait_for, TimeoutError
    setup()
    
    data = []
    
    @wait_for(timeout=200.0)
    def run_it_already():
        crawler = CrawlerRunner()
        return crawler.crawl(MySpider, 
                             start_urls_and_ids=[(company_id, convert_url(url)) for company_id,url in urls], 
                             output=data
                            )
    try:
        run_it_already()
    except TimeoutError:
        # In case of timeout, just return what we managed to scrape
        print("Timed out for given URLs", list(urls))
        print(data)

    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pandas_df = pd.read_csv("s3://onai-ml-dev-eu-west-1/web_crawler/data/seed_urls/company-urls.csv", header=0, sep='\t').astype(str)
tst_urls = pandas_df["url"].iloc[:100]

In [ ]:
%%time
data1 = run_spider(tst_urls)

In [ ]:
len(data1)

In [ ]:
for el in data1:
    if "http://www.thetacapital.com" in el[2]:
        print(el)

In [11]:
crawl_df = spark.createDataFrame(df
            .repartition(1000)
            .rdd
            .mapPartitions(lambda u: run_spider((el.company_id, el.url) for el in u))
            , T.StructType([
                T.StructField("company_id", T.StringType()),
                T.StructField("website", T.StringType()),
                T.StructField("description", T.StringType()),
                T.StructField("url", T.StringType()),
                T.StructField("page_title", T.StringType()),
                T.StructField("content_type", T.StringType()),
                T.StructField("raw_content", T.StringType())
            ])
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
crawl_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- company_id: string (nullable = true)
 |-- website: string (nullable = true)
 |-- description: string (nullable = true)
 |-- url: string (nullable = true)
 |-- page_title: string (nullable = true)
 |-- content_type: string (nullable = true)
 |-- raw_content: string (nullable = true)

In [13]:
(crawl_df
 .repartition(100)
 .write
 .parquet("s3://onai-ml-dev-eu-west-1/web_crawler/data/raw_crawl_results", mode="overwrite")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
crawl_results = spark.read.load("s3://onai-ml-dev-eu-west-1/web_crawler/data/raw_crawl_results")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
crawl_results.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1014614

In [16]:
crawl_results.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- company_id: string (nullable = true)
 |-- website: string (nullable = true)
 |-- description: string (nullable = true)
 |-- url: string (nullable = true)
 |-- page_title: string (nullable = true)
 |-- content_type: string (nullable = true)
 |-- raw_content: string (nullable = true)

In [20]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- company_id: integer (nullable = true)
 |-- company_name: string (nullable = true)
 |-- url: string (nullable = true)
 |-- country: string (nullable = true)

In [32]:
joined_companies = (crawl_results.join(df,
                                       on = ["company_id"]
                                      )
                    .drop(df.url)
                   )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
joined_companies.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1014614

In [36]:
joined_companies.write.parquet("s3://onai-ml-dev-eu-west-1/web_crawler/data/urls_and_content", mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
tst = spark.read.load("s3://onai-ml-dev-eu-west-1/web_crawler/data/urls_and_content")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
tst.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1014614

In [39]:
tst.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- company_id: string (nullable = true)
 |-- website: string (nullable = true)
 |-- description: string (nullable = true)
 |-- url: string (nullable = true)
 |-- page_title: string (nullable = true)
 |-- content_type: string (nullable = true)
 |-- raw_content: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- country: string (nullable = true)

In [ ]:
(tst
 .repartition(100)
 .write
 .parquet("s3://onai-ml-dev-eu-west-1/web_crawler/data/sample_urls_and_content", mode="overwrite")
)

## Notes
* 10 urls per partition ~ 2 mins per partition: takes about 1.5 hrs to finish 10k urls
* 100 urls per partition and config fixes: 21 minutes
* 1000 urls per partition: 17 mins
* looking only at main page and about, takes 8 mins 100 docs per partition